In [4]:
import cv2
import numpy as np 
from tqdm import tqdm
from matplotlib import pyplot as plt 

In [11]:
cap = cv2.VideoCapture(0)
cap2 = cv2.VideoCapture(3)

sift = cv2.xfeatures2d.SIFT_create()
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params, search_params)

if not cap.isOpened() or not cap2.isOpened():
    print("Cannot open camera")
    exit()
count = 0
while True:
    # Capture frame-by-frame
    ret, img1 = cap.read()
    ret2, img2 = cap2.read()
    
    keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
    keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)
    
    matches = bf.match(descriptors_1,descriptors_2)
    
    matches = flann.knnMatch(descriptors_1, descriptors_2, k=2)
    
    matchesMask = [[0, 0] for i in range(len(matches))]
    good = []
    pts1 = []
    pts2 = []

    for i, (m, n) in enumerate(matches):
        if m.distance < 0.70*n.distance:
            # Keep this keypoint pair
            matchesMask[i] = [1, 0]
            good.append(m)
            pts2.append(keypoints_2[m.trainIdx].pt)
            pts1.append(keypoints_1[m.queryIdx].pt)
            
    draw_params = dict(matchColor=(0, 255, 0),
                   singlePointColor=(255, 0, 0),
                   matchesMask=matchesMask[300:500],
                   flags=cv2.DrawMatchesFlags_DEFAULT)

    keypoint_matches = cv2.drawMatchesKnn(
        img1, keypoints_1, img2, keypoints_2, matches[300:500], None, **draw_params)
    keypoint_matches = cv2.resize(keypoint_matches, (1000,400))
    
    if len(pts1) >= 8:
        pts1 = np.int32(pts1)
        pts2 = np.int32(pts2)
        F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)
        # We select only inlier points
        pts1 = pts1[mask.ravel()==1]
        pts2 = pts2[mask.ravel()==1]
        print("saving image", len(pts1), count)
        cv2.imwrite("./best_pairs/"+str(count)+"L.jpg", img1)
        cv2.imwrite("./best_pairs/"+str(count)+"R.jpg", img2)
        count = count+1
        
    
    # # Display the resulting frame
    # cv2.imshow('Stereo 1', img1)
    cv2.imshow('best_pairs',keypoint_matches)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    if key == ord('s'):
        print("Saving image")
        cv2.imwrite("./best_pairs/"+str(count)+"L.jpg", img1)
        cv2.imwrite("./best_pair/"+str(count)+"R.jpg", img2)
        count = count+1
# When everything done, release the capture
cap.release()
cap2.release()
cv2.destroyAllWindows()

saving image 11 0
saving image 44 1
saving image 47 2
saving image 32 3
saving image 44 4
saving image 42 5
saving image 46 6
saving image 50 7
saving image 52 8
saving image 41 9
saving image 50 10
saving image 33 11
saving image 43 12
saving image 32 13
saving image 35 14
saving image 26 15
saving image 39 16
saving image 31 17
saving image 42 18
saving image 38 19
saving image 31 20
saving image 48 21
saving image 54 22
saving image 35 23
saving image 44 24
saving image 42 25
saving image 44 26
saving image 48 27
saving image 49 28
saving image 39 29
saving image 51 30
saving image 52 31
saving image 54 32
saving image 43 33
saving image 46 34
saving image 42 35
saving image 51 36
saving image 55 37
saving image 35 38
saving image 39 39
saving image 41 40
saving image 42 41
saving image 36 42
saving image 37 43
saving image 33 44
saving image 41 45
saving image 36 46
saving image 37 47
saving image 43 48
saving image 38 49
saving image 24 50
saving image 30 51
saving image 30 52
sav

In [6]:
cap.release()
cap2.release()
cv2.destroyAllWindows()